This notebook contains an early presentation of [Program synthesis draft PR](https://github.com/XanaduAI/pennylane-mlir/pull/343) 

Overview
-------

The sub-project currenntly contains three Python modules, defining a grammar of an imperative language, the pretty-prining code and an expression build tracker (see below).


In [1]:
from catalyst.synthesis import *

Grammar
----------

Located at [synthesis.grammar](https://github.com/XanaduAI/pennylane-mlir/blob/progam_synthesis/frontend/catalyst/synthesis/grammar.py#L1)
The definitions specify a simple untyped imperative language with quantum specific in mind:
* Function declarations can specify number of quantum wires or the name of quantum device
* Control flow statements have `style` field, which could be set to `Python`, `Catalyst` or `JAX` (the printer only supports the `Python` style right now)

Some example definitions:

In [2]:
asgn = AssignStmt(VName("var"), FCallExpr(FName("func"), [VRefExpr(VName("a"))]))
print(asgn)

AssignStmt(vname=VName(val='var'), expr=FCallExpr(fname=FName(val='func'), args=[VRefExpr(vname=VName(val='a'))]))


In [3]:
loop = ForLoopStmt(VName('i'),
                   ConstExpr(0),
                   ConstExpr(100),
                   POIStmt([asgn]), ControlFlowStyle.Python)
print(loop)

ForLoopStmt(loopvar=VName(val='i'), lbound=ConstExpr(val=0), ubound=ConstExpr(val=100), body=POIStmt(stmts=[AssignStmt(vname=VName(val='var'), expr=FCallExpr(fname=FName(val='func'), args=[VRefExpr(vname=VName(val='a'))]))]), style=<ControlFlowStyle.Python: 0>)


In [4]:
fndef = FDefStmt(FName("main"),
                 [VName("a"),VName("b")],
                 POIStmt([loop]),
                 qwires=2)
print(fndef)

FDefStmt(fname=FName(val='main'), args=[VName(val='a'), VName(val='b')], body=POIStmt(stmts=[ForLoopStmt(loopvar=VName(val='i'), lbound=ConstExpr(val=0), ubound=ConstExpr(val=100), body=POIStmt(stmts=[AssignStmt(vname=VName(val='var'), expr=FCallExpr(fname=FName(val='func'), args=[VRefExpr(vname=VName(val='a'))]))]), style=<ControlFlowStyle.Python: 0>)]), qwires=2, qdevice=None)


Pretty Printer
--------------

The pretty printer essentially provides a collection of `pstr_*` and the `pprint` top-level functions. The latter knows how to print any grammar. 

In [5]:
pprint(asgn)

var = func(a)


In [6]:
pprint(loop)

for i in range(int(0), int(100)):
    var = func(a)


In [7]:
pprint(fndef)

@qml.qnode(qml.device("catalyst-lightning", wires=2))
def main(a, b):
    for i in range(int(0), int(100)):
        var = func(a)


Expression Build Tracker
-------------------------

While the modules described above already provide basic tools for building random programs, we also define an build helper which does the following
* Maintains the set of availabe "Points of Insertion".
* For each Point of Insertion, tracks the context which currently includes the following information:
  - What variables are visible from this POI.
  - For quantum functions, how many wires did we declare.
  
Essentially, the idea of Tracker is to separate certain stateful parts from the rest of the program generation logic, allowing stateless algorithms. 

Any expression could be loaded into the Tracker and benefit from its interface

In [8]:
t = track(fndef)

print(len(t.pois))

2


Pretty-printer is avare of Trackers. By default, it includes the information about the active points of insertions.

In [9]:
pprint(t)

@qml.qnode(qml.device("catalyst-lightning", wires=2))
def main(a, b):
    for i in range(int(0), int(100)):
        var = func(a)
        # poi 140678570068192
        # a, b, i, var
    # poi 140678570081776
    # a, b


Below we create a conditional statement and include it into the first point of insertion

In [10]:
cond = CondStmt(trueExpr, POIStmt(),POIStmt(), ControlFlowStyle.Python)
t.insert_statement(1,cond)
print(f"#POIs after inserting cond is {len(t.pois)}")
t.insert_statement(3,asgn)
pprint(t)

#POIs after inserting cond is 4
@qml.qnode(qml.device("catalyst-lightning", wires=2))
def main(a, b):
    for i in range(int(0), int(100)):
        var = func(a)
        # poi 140678570068192
        # a, b, i, var
    if bool(True) then:
        pass
        # poi 140678570069488
        # a, b
    else:
        var = func(a)
        # poi 140678570070640
        # a, b, var
    # poi 140678570081776
    # a, b


Conclusion
------

Later plans include:

* Completig the Catalyst-version of the control flow pretty-printing
* Writing a parametrized random `FCallExpr` generator for PennyLane gate calls.
* Implementing a couple of generator algorithms which would:
  - Keep track of some kind of budget to spend on certain operations.
  - Make use of the above Tracker API to spend their budget to filling the function body.
  